# A faire
**Idées indicateurs :**
- Périodes des candidatures
- taux de candidatures en fonction du nombre de consultation par métier

**Ajouter tenant id matching dispo sur le site admi pour identifier la source des consultations**

Discussion avec Christine et Elodie
- Nb d'annonce diffusées par opérateur par mois
- Durée moyenne de publication d'annonce par opérateur
- Nombre de réponse par opérateurs et par mois (nombre de clics sur chaque candidatures)
- attractivité des offres selon l'opportunité (regroupement type d'offre)
- nom de domaine sur qui publie
- bijection splru, keolis et ratp vers la meme chose (passer via tenant id...)
- pouvoir sélectionner les dates à un moment

Attention il y'a des offres publiées en permanence

Fonctionnement ouverture fermeture des annonces : 3 systèmes

SPLRU : empowill



In [11]:
# Import libraries
import pandas as pd
import plotly.express as px

In [12]:
# Import fichiers
df = pd.read_csv("stats_evenements_2025-04-17_11-25-16.csv", encoding="ISO-8859-1",sep=';')

# Colonnes df
columns = df.columns

# Format df
shape = df.shape

# id_offre
id_offre = sorted(df['Id Offre'].unique())
print(len(id_offre))

# add a "1" column
df['compteur'] = 1

print(df.head())

304
   Événement ID              Date Événement Code Événement Libellé Événement  \
0          6416  2025-04-17T10:37:16.731898            CST      Consultation   
1          6415  2025-04-17T10:36:42.465018            CST      Consultation   
2          6414  2025-04-17T10:36:10.047795            CDT       Candidature   
3          6413  2025-04-17T10:36:09.531375            CDT       Candidature   
4          6412  2025-04-17T10:35:46.618461            CST      Consultation   

   Id Offre                                        Titre Offre  \
0       299  Conducteur Tramway avec des missions de contrô...   
1       300                            Technicien méthodes F/H   
2       289                            Chefs de Bord DCI - H/F   
3       289                            Chefs de Bord DCI - H/F   
4       289                            Chefs de Bord DCI - H/F   

       Date Publication Offre  Offre publiée ? Code Domaine  \
0  2025-04-09T18:05:48.072937             True         

In [13]:
# Nombre d'événements par type
events_by_type = df['Libellé Événement'].value_counts().reset_index()
events_by_type.columns = ['Type d\'événement', 'Nombre d\'événements']

fig = px.bar(events_by_type, 
             x='Type d\'événement', 
             y='Nombre d\'événements', 
             title='Nombre d\'événements par type', 
             color='Type d\'événement', 
             text='Nombre d\'événements')

fig.update_layout(xaxis_title='Type d\'événement', 
                  yaxis_title='Nombre d\'événements', 
                  xaxis=dict(tickangle=45))
fig.show()

In [14]:
# Calculate the number of job offers by domain
job_offers_by_domain = df.groupby('Libellé Domaine').size().reset_index(name='count')

# Visualize the number of job offers by domain using Plotly
fig = px.bar(job_offers_by_domain, 
             x='Libellé Domaine', 
             y='count', 
             title="Nombre d'offres d'emploi par domaine", 
             labels={'Libellé Domaine': 'Domaine', 'count': "Nombre d'offres d'emploi"}, 
             color='Libellé Domaine', 
             text='count')

fig.update_layout(xaxis_title='Domaine', 
                  yaxis_title="Nombre d'offres d'emploi", 
                  xaxis=dict(tickangle=45))
fig.show()


In [16]:
# Count the number of operations per offer
event_count_per_offer = df['Id Offre'].value_counts()
print(event_count_per_offer.describe())

# # Extract Id Offre with the highest count of Code Événement
# id_offre_highest_event_count = event_count_per_offer.idxmax()

# # Filter out rows with Id Offre == id_offre_highest_event_count and store them in df_temp
# df_temp = df[df['Id Offre'] == id_offre_highest_event_count]

# Remove rows with Id Offre == id_offre_highest_event_count from the original dataframe
# df = df[df['Id Offre'] != id_offre_highest_event_count]

# Recount the number of operations per offer
event_count_per_offer = df['Id Offre'].value_counts()

# Box plot for event count per offer using Plotly Express
fig = px.box(event_count_per_offer, orientation='h', title='Box Plot of Event Count per Offer')
fig.update_layout(xaxis_title='Event Count')
fig.show()

count    304.000000
mean      21.095395
std       61.977059
min        1.000000
25%        3.000000
50%        4.000000
75%       17.000000
max      948.000000
Name: count, dtype: float64


In [ ]:
df_temp = df.groupby(['Id Offre', 'Libellé Événement'])['compteur'].sum().reset_index()
print(df_temp)

# Drop Id Offre = 245 => 800 opérations
# df_temp = df_temp[df_temp['Id Offre'] != 245]

fig = px.bar(df_temp, 
             x='Id Offre', 
             y='compteur', 
             color='Libellé Événement', 
             title='Event Count per Offer by Event Type')
fig.update_layout(xaxis_title='Id Offre', yaxis_title='Event Count')
fig.show()

      Id Offre             Libellé Événement  compteur
0            1                  Consultation         5
1            1  Dépublication offre d'emploi         1
2            1      Insertion offre d'emploi         1
3            1    Publication offre d'emploi         1
4            2                  Consultation         2
...        ...                           ...       ...
1109       304    Publication offre d'emploi         1
1110       305                   Candidature         1
1111       305                  Consultation         7
1112       305      Insertion offre d'emploi         1
1113       305    Publication offre d'emploi         1

[1114 rows x 3 columns]


In [ ]:
# Convert Date Événement to datetime
df['Date Événement'] = pd.to_datetime(df['Date Événement'])

# Extract month and year from the 'Date Événement' column and create a new columns 'Month' and 'Year'
df['Month'] = df['Date Événement'].dt.month
df['Year'] = df['Date Événement'].dt.year

# Map month numbers to month names
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
               7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

for year in [2024, 2025]:
    df_temp = df[df['Year'] == year].groupby(['Month', 'Libellé Événement'])['compteur'].sum()
    fig = px.bar(df_temp.reset_index(), 
                 x=df_temp.reset_index()['Month'].map(month_names), 
                 y='compteur', 
                 color='Libellé Événement', 
                 title=f'Event Count per Month by Event Type and Year - {year}')
    fig.update_layout(xaxis_title=None, yaxis_title='Event Count')
    fig.show()


In [ ]:
# Calculer le nombre de consultations (CST) et de candidatures (CDT)
num_consultations = df[df['Code Événement'] == 'CST'].shape[0]
num_candidatures = df[df['Code Événement'] == 'CDT'].shape[0]

# Calculer le taux de candidatures en fonction du nombre de consultations
candidature_rate = (num_candidatures / num_consultations) * 100

print(f"Taux de candidatures en fonction du nombre de consultations: {candidature_rate:.2f}%")


Taux de candidatures en fonction du nombre de consultations: 41.04%


In [ ]:
# Calculer le taux de candidatures par code domaine
domain_candidature_rates = {}

# Obtenir les codes domaines uniques
unique_domains = df['Code Domaine'].unique()

for domain in unique_domains:
    domain_data = df[df['Code Domaine'] == domain]
    # Calculer le nombre de consultations (CST) et de candidatures (CDT) pour le domaine
    num_consultations = domain_data[domain_data['Code Événement'] == 'CST'].shape[0]
    num_candidatures = domain_data[domain_data['Code Événement'] == 'CDT'].shape[0]
    # Calculer le taux de candidatures en fonction du nombre de consultations pour le domaine
    if num_consultations > 0:
        candidature_rate = (num_candidatures / num_consultations) * 100
    else:
        candidature_rate = 0
        
    domain_candidature_rates[domain] = candidature_rate

# Afficher les taux de candidatures par code domaine
for domain, rate in domain_candidature_rates.items():
    print(f"Taux de candidatures en fonction du nombre de consultations pour le domaine {domain}: {rate:.2f}%")

# Visualize the candidature rates by domain
fig = px.bar(x=domain_candidature_rates.keys(), 
                y=domain_candidature_rates.values(), 
                labels={'x': 'Code Domaine', 'y': 'Candidature Rate (%)'}, 
                title='Candidature Rates by Domain')
fig.update_layout(yaxis_title='Candidature Rate (%)', xaxis_title='Code Domaine')
fig.show()


Taux de candidatures en fonction du nombre de consultations pour le domaine ML: 61.01%
Taux de candidatures en fonction du nombre de consultations pour le domaine BT: 31.05%
Taux de candidatures en fonction du nombre de consultations pour le domaine RU: 14.77%


In [ ]:
df['date'] = df['Date Événement'].dt.date
df_temp = df.groupby(['date', 'Libellé Événement'])['compteur'].sum().reset_index()

# Créer une visualisation de la répartition des événements au fil du temps avec distinction libellé événements
fig = px.bar(df_temp, 
             x='date', 
             y='compteur', 
             color='Libellé Événement', 
             barmode='group',
             title='Répartition des événements au fil du temps avec distinction libellé événements')

# Afficher la figure
fig.show()

In [ ]:
df_temp = df.loc[df['Id Offre'] == 245]

df_temp['Tenant ID'] = df_temp['Tenant ID'].fillna("inc")

df_temp = df_temp.groupby(['date','Libellé Événement','Tenant ID'])['compteur'].sum()

fig = px.bar(df_temp.reset_index(), 
             x='date', 
             y='compteur', 
             color='Libellé Événement', 
             barmode='group', 
             title='Event Count by Date and Event Type for Id Offre 245')
fig.update_layout(xaxis_title='Date', yaxis_title='Event Count')
fig.show()

C:\Users\VBO\AppData\Local\Temp\ipykernel_29948\3101969297.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Mapping des Tenant ID vers les courriels
tenant_courriel_mapping = {
    'b87cc266-09c4-40cc-8dfa-c92e08bf9cb4': 'anonyme@ratpdev.com',
    '7124e463-2734-41bf-bddb-3e475374f94c': 'anonyme@keolis-lyon.fr',
    '6b23e274-b621-4690-a6c9-bf8828efd33e': 'anonyme@mobilites-lyonnaises.fr',
    '846ece69-a6d3-4892-83ca-a966df6f640e': 'anonyme@sytral.fr'
}

# Créer une nouvelle colonne 'Courriel' dans le DataFrame df
df['courriel'] = df['Tenant ID'].map(tenant_courriel_mapping)

In [26]:
# Nombre d'annonces diffusées par opérateur par mois
df_temp = df.loc[df['Libellé Événement'] == 'Publication offre d\'emploi']

# Convert Date Événement to datetime
df_temp['Date Événement'] = pd.to_datetime(df_temp['Date Événement'])

# Extract month and year from the 'Date Événement' column and create a new columns 'Month' and 'Year'
df_temp['Month'] = df_temp['Date Événement'].dt.month
df_temp['Year'] = df_temp['Date Événement'].dt.year

grouped = df_temp.groupby(['Year','Month', 'Code Opérateur']).size().reset_index(name='count')
print(grouped)

for year in grouped['Year'].unique():
    yearly_data = grouped[grouped['Year'] == year]
    fig = px.bar(yearly_data, 
                 x='Month', 
                 y='count', 
                 color='Code Opérateur', 
                 barmode='group', 
                 title=f"Nombre d'annonces diffusées par opérateur par mois - {year}",
                 labels={'Month': 'Mois', 'count': "Nombre d'annonces", 'Code Opérateur': 'Opérateur'})
    fig.update_layout(xaxis_title='Mois', yaxis_title="Nombre d'annonces", xaxis=dict(tickmode='linear'))
    fig.show()

    Year  Month Code Opérateur  count
0   2024     10          KEOLY      4
1   2024     10         RATPLY     96
2   2024     11          KEOLY      6
3   2024     11         RATPLY     60
4   2024     12          KEOLY      4
5   2024     12         RATPLY      3
6   2025      1          KEOLY     28
7   2025      1         RATPLY     23
8   2025      1             RU     24
9   2025      2          KEOLY      1
10  2025      2         RATPLY      5
11  2025      2             RU      5
12  2025      3         RATPLY     12
13  2025      3             RU      7
14  2025      4          KEOLY      5
15  2025      4         RATPLY     10
16  2025      4             RU      3


C:\Users\VBO\AppData\Local\Temp\ipykernel_30108\1663526360.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\VBO\AppData\Local\Temp\ipykernel_30108\1663526360.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\VBO\AppData\Local\Temp\ipykernel_30108\1663526360.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs